**Adaptive-Prompt**, a method that selects the most informative examples (called exemplars) to guide a large language model (LLM) like GPT-4 in solving reasoning tasks. The goal is to iteratively build a set of exemplars that maximize the model’s performance on new, unseen questions by adaptively choosing which examples to add based on model uncertainty.

In [3]:
# INSTALLING REQUIRED LIBRARIES
!pip install tqdm openai pandas dotenv



* **Datasets:** https://github.com/ad-freiburg/large-qa-datasets?tab=readme-ov-file





In [42]:
# PREPARING TRAINING DATA

import json

with open("/content/webquestions.examples.train.json") as f_in:
  d_1 = json.load(f_in)

with open("/content/free917.train.examples.canonicalized.json") as f1_in:
  d_2 = json.load(f1_in)

questions = []
for i in d_1:
  entry = {
      "url" : i["url"],
      "answers" : i["targetValue"],
      "question" : i["utterance"]
  }
  questions.append(entry["question"])

for i in d_2:
  entry = {
      "question" : i["utterance"],
      "answers" : i["targetFormula"]
  }
  questions.append(entry["question"])

with open("/content/questions_train.json","w") as f_out:
  json.dump(questions, f_out)

In [33]:
import json

with open("/content/questions_train.json", "r") as f:
  questions_train = json.load(f)

print(questions_train[:10])

['what is the name of justin bieber brother?', 'what character did natalie portman play in star wars?', 'what state does selena gomez?', 'what country is the grand bahama island in?', 'what kind of money to take to bahamas?', 'what character did john noble play in lord of the rings?', 'who does joakim noah play for?', 'where are the nfl redskins from?', 'where did saki live?', 'how old is sacha baron cohen?']


In [34]:
# PREPARING TESTING DATA

import json

with open("/content/webquestions.examples.test.json") as f:
  d_1 = json.load(f)

with open("/content/free917.test.examples.canonicalized.json") as f1:
  d_2 = json.load(f1)

questions = []
for i in d_1:
  entry = {
      "url" : i["url"],
      "answers" : i["targetValue"],
      "question" : i["utterance"]
  }
  questions.append(entry["question"])

for i in d_2:
  entry = {""
      "question" : i["utterance"],
      "answers" : i["targetFormula"]
  }
  questions.append(entry["question"])

with open("/content/questions_test.json","w") as fOut:
  json.dump(questions, fOut)

In [6]:
import openai
import random
from tqdm import tqdm
import json
import pandas as pd

In [36]:
api_key = "fill-in-the-api-key-you-get-from-open-ai"

In [37]:
# Reference 1: https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/
client = openai.OpenAI(api_key=api_key)


In [38]:
def buildPrompt(exemplers, question):
  prompt = ""
  for ex in exemplers:
    prompt += f"""
    Question: {ex["question"]} \n
    Reasoning: {ex["reasoning"]} \n
    Answer: {ex["answers"]} \n\n
    """
  prompt += f"Question: {question} \nAnswer:"
  return prompt

In [39]:
def computeUncertainity(exemplers, question, l):
  prompt = buildPrompt(exemplers, question)
  answers = []
  for i in range(l):
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [{"role":"user",
                     "content":prompt}],
        temperature = 0.7
    )
    answers.append(response.choices[0].message.content.strip())
  unique_answers = list(set(answers))
  return (len(unique_answers) / l)

In [40]:
def adaptivePrompting(questions, k, l):
  exemplers = []
  remaining_questions = questions.copy()
  for i in range(k):
    uncertainity = []
    for j in tqdm(remaining_questions, desc = "evaluating uncertainity...."):
      u = computeUncertainity(exemplers,j,l)
      uncertainity.append((j,u))

    ques_selected, max_u = max(uncertainity, key=lambda x: x[1])
    print(f"\nSelected question (highest uncertainty = {max_u:.2f}):\n{ques_selected['question']}")
    reasoning = input(f"Please provide step-by-step reasoning for:\n'{ques_selected['question']}'\nReasoning: ")
    answer = input(f"Please provide the correct answer for:\n'{ques_selected['question']}'\nAnswer: ")
    exemplers.append({
        "question": ques_selected["question"],
        "reasoning": reasoning,
        "answers": answer
        })
    remaining_questions.remove(ques_selected)
  return exemplers

In [41]:
'''
The temperature parameter controls the randomness of the model's output.
A value of 0 makes the output more deterministic (less random),
while higher values make it more creative and varied.

max_tokens: This sets the maximum number of tokens (words or sub-word
units) that the model will generate in its response.
'''

K = 3 # No. of exemplers
L = 5 # No. of times we run loop to determine the uncertainity

train_data_path = "/content/questions_train.json"
test_data_path = "/content/questions_test.json"

if __name__ == "__main__":
  print("""
  The Power of Adaptation: \n\n
  Boosting In-Context Learning through Adaptive Prompting
  """)
  with open(train_data_path, "r") as f:
    train_data = json.load(f)

  with open(test_data_path, "r") as f:
    test_data = json.load(f)

  exemplers = adaptivePrompting(train_data, K, L)
  for ex in exemplers:
    print(f"Question: {ex['question']}\nReasoning: {ex['reasoning']}\nAnswer: {ex['answers']}\n")
  print("Verifying model understanding with test set")
  for q in test_data:
    prompt = buildPrompt(exemplers, q)
    response = client.ChatCompletion.create(
        model = "gpt-4",
        messages = [{"role":"user",
                     "content":prompt}],
        temperature = 0
    )
    print(f"Question: {q}\nAnswer: {response['choices'][0]['message']['content'].strip()}\n")


  The Power of Adaptation: 


  Boosting In-Context Learning through Adaptive Prompting
  


evaluating uncertainity....:   0%|          | 0/4419 [00:02<?, ?it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Since there's a limit on the number of calls you can make with the free version we couldn't test the model.